###  인공신경망 모델 살펴보기
- 구성 : 입력층 + 은닉층 + 출력층
    * 입력층 : 입력값 <- 입력 피쳐 수 / 출력값 <- 층의 퍼셉트론 수
    * 출력층 : 입력값 <- 이전층의 결과 수, 출력값 <- 입력 타겟 수

In [14]:
import torch
import torch.nn.functional as F
import torch.nn as nn

import torch.random as random
random.manual_seed(12)

In [15]:
# 데이터 생성
# 피쳐 : 점수 공부시간
# 타겟 : 합격 불합격(이진)
featureTS = torch.FloatTensor([[40, 4], [60, 5], [80, 6], [100, 8]])
targetTS = torch.LongTensor([[0], [1], [1], [1]])

print(f"[featureTS] {featureTS.shape}, {featureTS.ndim}D")
print(f"[targetTS] {targetTS.shape}, {targetTS.ndim}D")

[featureTS] torch.Size([4, 2]), 2D
[targetTS] torch.Size([4, 1]), 2D


In [16]:
## ANN 모델
class Net(nn.Module):
    
    # 모델 구조 설정
    def __init__(self):
        super().__init__()
        # 입력층 input 2는 고정
        self.layer1 = nn.Linear(2, 4)
        self.sig1 = nn.Sigmoid() # 재탕 가능 
        
        # 은닉층
        self.layer2 = nn.Linear(4, 4)
        self.sig2 = nn.Sigmoid()
        
        # 출력층 out 1은 고정
        self.layer3 = nn.Linear(4, 1)
        self.sig3 = nn.Sigmoid()
    
    # 순전파 함수
    def forward(self, x):
        x = self.layer1(x)
        x = self.sig1(x)
        x = self.layer2(x)
        x = self.sig2(x)
        x = self.sig3(x)
        x = self.layer3(x)
        
        return x
        
        

In [17]:
# 모델 인스턴스 생성 
model1 = Net()
model1

Net(
  (layer1): Linear(in_features=2, out_features=4, bias=True)
  (sig1): Sigmoid()
  (layer2): Linear(in_features=4, out_features=4, bias=True)
  (sig2): Sigmoid()
  (layer3): Linear(in_features=4, out_features=1, bias=True)
  (sig3): Sigmoid()
)

In [18]:
# 파라미터 보고 싶을 때
for param in model1.parameters():
    print(param)

Parameter containing:
tensor([[-0.0485, -0.3779],
        [-0.0669,  0.1232],
        [-0.1292, -0.5273],
        [ 0.1941, -0.3648]], requires_grad=True)
Parameter containing:
tensor([ 0.3270,  0.3146, -0.4253,  0.2755], requires_grad=True)
Parameter containing:
tensor([[ 0.0830,  0.1318,  0.0559, -0.3738],
        [ 0.4790,  0.3443, -0.3744, -0.0544],
        [ 0.1601, -0.4446, -0.3427,  0.3137],
        [ 0.2216, -0.2283, -0.1997,  0.1099]], requires_grad=True)
Parameter containing:
tensor([ 0.0784,  0.1083, -0.0661,  0.3813], requires_grad=True)
Parameter containing:
tensor([[-0.1784, -0.2396, -0.2434, -0.3128]], requires_grad=True)
Parameter containing:
tensor([0.1423], requires_grad=True)


In [19]:
### 학습 진행
## 에포크 : 데이터 처음부터 끝까지 한번 학습
## 배치사이즈 : 데이터를 일정한 크기로 자른 것

output = model1(featureTS)
output

tensor([[-0.4739],
        [-0.4738],
        [-0.4738],
        [-0.4738]], grad_fn=<AddmmBackward0>)

In [25]:
### 손실 계산 -> 0.5기준
### 정확도(acc)
correct = sum((output>0.5) == targetTS)
accuracy = correct/targetTS.shape[0]

In [26]:
print(f"accuracy: {accuracy.item()}")

accuracy: 0.25


In [30]:
## 정밀도

correct = sum(output>0.5)
if not correct:
    print("정밀도 : 0")

정밀도 : 0


In [28]:
## 재현율
correct = sum((output>0.5) == targetTS)
correct

tensor([1])

# 손실함수 예측과 정답 사이의 오차
- 회귀 : MSE, MAE, RMSE
- 분류 : binary_cross_entropy, cross_entropy

In [33]:
print(f"targetTS => \n{targetTS}")
print(f"output => \n{output}")

targetTS => 
tensor([[0],
        [1],
        [1],
        [1]])
output => 
tensor([[-0.4739],
        [-0.4738],
        [-0.4738],
        [-0.4738]], grad_fn=<AddmmBackward0>)


In [34]:
F.binary_cross_entropy(output, targetTS) # 어떤 것들은 확률로 넣어야함, cross_entropy는 지가 알아서 확률을 결과로 변환힘

RuntimeError: Found dtype Long but expected Float